<a href="https://colab.research.google.com/github/Ceazy-Gentleman/practice-/blob/master/TimeSeriesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装并载入Gluonts工具包
示例中gluonts.trainer 变为了 gluonts.mx.trainer

In [ ]:
pip list

In [ ]:
pip install mxnet

In [ ]:
pip install gluonts

In [ ]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
estimator = DeepAREstimator(freq="5min",prediction_length=36,trainer=Trainer(epochs=10))

# 内置的数据集
In general, the datasets provided by GluonTS are objects that consists of three main members:

- dataset.train is an iterable collection of data entries used for training. Each entry corresponds to one time series
 
- dataset.test is an iterable collection of data entries used for inference. The test dataset is an extended version of the train dataset that contains a window in the end of each time series that was not seen during training. This window has length equal to the recommended prediction length.

- dataset.metadata contains metadata of the dataset such as the frequency of the time series, a recommended prediction horizon, associated features, etc.

In [ ]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas
print(f"Available datasets: {list(dataset_recipes.keys())}")

In [ ]:
dataset = get_dataset("m4_hourly", regenerate=True)

In [ ]:
entry = next(iter(dataset.train))
train_series = to_pandas(entry)
train_series.plot()
plt.grid(which="both")
plt.legend(["train series"], loc="upper left")
plt.show()

In [ ]:
entry = next(iter(dataset.test))
test_series = to_pandas(entry)
test_series.plot()
plt.axvline(train_series.index[-1], color='r') # end of train dataset
plt.grid(which="both")
plt.legend(["test series", "end of train series"], loc="upper left")
plt.show()

In [ ]:
print(f"Length of forecasting window in test dataset: {len(test_series) - len(train_series)}")
print(f"Recommended prediction horizon: {dataset.metadata.prediction_length}")
print(f"Frequency of the time series: {dataset.metadata.freq}")

# 自定义数据集
At this point, it is important to emphasize that GluonTS does not require this specific format for a custom dataset that a user may have. The only requirements for a custom dataset are to be iterable and have a “target” and a “start” field. To make this more clear, assume the common case where a dataset is in the form of a numpy.array and the index of the time series in a pandas.Timestamp (possibly different for each time series):

In [ ]:
N = 10  # number of time series
T = 100  # number of timesteps
prediction_length = 24
freq = "1H"
custom_dataset = np.random.normal(size=(N, T))
start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

Now, you can split your dataset and bring it in a GluonTS appropriate format with just two lines of code:

In [ ]:
from gluonts.dataset.common import ListDataset
# train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
train_ds = ListDataset(
    [{'target': x, 'start': start} for x in custom_dataset[:, :-prediction_length]],
    freq=freq
)
# test dataset: use the whole dataset, add "target" and "start" fields
test_ds = ListDataset(
    [{'target': x, 'start': start} for x in custom_dataset],
    freq=freq
)

# 模型调用方法
GluonTS comes with a number of pre-built models. All the user needs to do is configure some hyperparameters. The existing models focus on (but are not limited to) probabilistic forecasting. Probabilistic forecasts are predictions in the form of a probability distribution, rather than simply a single point estimate.

We will begin with GulonTS’s pre-built feedforward neural network estimator, a simple but powerful forecasting model. We will use this model to demonstrate the process of training a model, producing forecasts, and evaluating the results.

GluonTS’s built-in feedforward neural network (SimpleFeedForwardEstimator) accepts an input window of length context_length and predicts the distribution of the values of the subsequent prediction_length values. In GluonTS parlance, the feedforward neural network model is an example of Estimator. In GluonTS, Estimator objects represent a forecasting model as well as details such as its coefficients, weights, etc.

In general, each estimator (pre-built or custom) is configured by a number of hyperparameters that can be either common (but not binding) among all estimators (e.g., the prediction_length) or specific for the particular estimator (e.g., number of layers for a neural network or the stride in a CNN).

Finally, each estimator is configured by a Trainer, which defines how the model will be trained i.e., the number of epochs, the learning rate, etc.

In [ ]:
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.mx import Trainer
estimator = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[10],
    prediction_length=dataset.metadata.prediction_length,
    context_length=100,
    freq=dataset.metadata.freq,
    trainer=Trainer(
        ctx="cpu",
        epochs=5,
        learning_rate=1e-3,
        num_batches_per_epoch=100
    )
)

After specifying our estimator with all the necessary hyperparameters we can train it using our training dataset dataset.train by invoking the train method of the estimator. The training algorithm returns a fitted model (or a Predictor in GluonTS parlance) that can be used to construct forecasts.

In [ ]:
predictor = estimator.train(dataset.train)


# 可视化及评估预测结果
With a predictor in hand, we can now predict the last window of the dataset.test and evaluate our model’s performance.

GluonTS comes with the make_evaluation_predictions function that automates the process of prediction and model evaluation. Roughly, this function performs the following steps:

- Removes the final window of length prediction_length of the dataset.test that we want to predict

- The estimator uses the remaining data to predict (in the form of sample paths) the “future” window that was just removed

- The module outputs the forecast sample paths and the dataset.test (as python generator objects)

In [ ]:
from gluonts.evaluation import make_evaluation_predictions
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

First, we can convert these generators to lists to ease the subsequent computations.

In [ ]:
forecasts = list(forecast_it)
tss = list(ts_it)

We can examine the first element of these lists (that corresponds to the first time series of the dataset). Let’s start with the list containing the time series, i.e., tss. We expect the first entry of tss to contain the (target of the) first time series of dataset.test.

In [ ]:
# first entry of the time series list
ts_entry = tss[0]
# first 5 values of the time series (convert from pandas to numpy)
print(np.array(ts_entry[:5]).reshape(-1,))
# first entry of dataset.test
dataset_test_entry = next(iter(dataset.test))
# first 5 values
dataset_test_entry['target'][:5]

The entries in the forecast list are a bit more complex. They are objects that contain all the sample paths in the form of numpy.ndarray with dimension (num_samples, prediction_length), the start date of the forecast, the frequency of the time series, etc. We can access all these information by simply invoking the corresponding attribute of the forecast object.

In [ ]:

# first entry of the forecast list
forecast_entry = forecasts[0]
print(f"Number of sample paths: {forecast_entry.num_samples}")
print(f"Dimension of samples: {forecast_entry.samples.shape}")
print(f"Start date of the forecast window: {forecast_entry.start_date}")
print(f"Frequency of the time series: {forecast_entry.freq}")

We can also do calculations to summarize the sample paths, such computing the mean or a quantile for each of the 48 time steps in the forecast window.

In [ ]:
print(f"Mean of the future window:\n {forecast_entry.mean}")
print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.5)}")

Forecast objects have a plot method that can summarize the forecast paths as the mean, prediction intervals, etc. The prediction intervals are shaded in different colors as a “fan chart”.

In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = 150
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()

In [ ]:
plot_prob_forecasts(ts_entry, forecast_entry)

We can also evaluate the quality of our forecasts numerically. In GluonTS, the Evaluator class can compute aggregate performance metrics, as well as metrics per time series (which can be useful for analyzing performance across heterogeneous time series).

我们也可以用数字来评估我们的预测质量。在GluonTS中，评估器类可以计算总的性能指标，以及每个时间序列的指标（这对于分析不同时间序列的性能非常有用）。

In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.test))

- Aggregate metrics are aggregated both across time-steps and across time series.

In [ ]:
import json
print(json.dumps(agg_metrics, indent=4))

- Individual metrics are aggregated only across time-steps.

In [ ]:
item_metrics.tail()

In [ ]:
item_metrics.plot(x='MSIS', y='MASE', kind='scatter')
plt.grid(which="both")
plt.show()

# 时序预测案例
## 导入数据集
  使用一个记录了提到 AMZN 股票代号的推特数量的公开数据集，用 pandas 下载并显示这个数据集

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
url = "https://raw.githubusercontent.com/numenta/NAB/master/data/realTweets/Twitter_volume_AMZN.csv"
df = pd.read_csv(url, header=0, index_col=0)

df[:200].plot(figsize=(20, 5), linewidth=2)
plt.grid()
plt.legend(["observations"])
plt.show()

## 划分数据集
GluonTS 提供了 Dataset 抽象层来统一各种不同输入格式的读取。在这里，我们使用 ListDataset 来读取在内存中以字典列表形式存储的数据。在 GluonTS 中，任何 Dataset 对象都是一个将字符串键值映射到任意值的字典的迭代器。 将数据截断到 2015 年 4 月 5 日为止，用于训练模型。在 4 月 5 日之后的数据将被用于测试训练好的模型。

In [ ]:
from gluonts.dataset.common import ListDataset
training_data = ListDataset(
    [{"start": df.index[0], "target": df.value[:"2015-04-05 00:00:00"]}],
    freq = "5min"
)

## 导入训练集进行训练
数据集在手，现在就可以使用刚才建立的 Estimator 的 train 接口来训练模型。当训练完成之后，你就会得到一个可以进行预测的 Predictor 对象。

In [ ]:
predictor = estimator.train(training_data=training_data)

## 评估模型
现在你可以使用 Predictor 来画出模型对于在训练数据之后的时间段的一些预测。绘出模型给出的预测有助于我们对这个模型的预测质量有一个定性的感受。 现在，基于同样的数据集，在之前用于训练的时间段之后的时间段取出若干组测试数据。如下图所示，模型给出的是概率预测，这是很重要的一点，因为概率预测提供了对于模型置信度的估计，并且可以使得下游基于此预测的决策能够考虑到预测的不确定性。

In [ ]:
test_data = ListDataset(
    [
        {"start": df.index[0], "target": df.value[:"2015-04-10 03:00:00"]},
        {"start": df.index[0], "target": df.value[:"2015-04-15 18:00:00"]},
        {"start": df.index[0], "target": df.value[:"2015-04-20 12:00:00"]}
    ],
    freq = "5min"
)


In [ ]:
from itertools import islice
from gluonts.evaluation.backtest import make_evaluation_predictions

def plot_forecasts(tss, forecasts, past_length, num_plots):
    for target, forecast in islice(zip(tss, forecasts), num_plots):
        ax = target[-past_length:].plot(figsize=(12, 5), linewidth=2)
        forecast.plot(color='g')
        plt.grid(which='both')
        plt.legend(["observations", "median prediction", "90% confidence interval", "50% confidence interval"])
        plt.show()

forecast_it, ts_it = make_evaluation_predictions(test_data, predictor=predictor, num_samples=100)
forecasts = list(forecast_it)
tss = list(ts_it)
plot_forecasts(tss, forecasts, past_length=150, num_plots=3)

看起来预测还算准确！现在我们可以定量地使用一系列指标来定量评估预测的质量。GluonTS 提供了用于评估模型的 Evaluator 模块。Evaluator 模块提供了常用的误差指标，例如 MSE、MASE、symmetric MAPE、RMSE 以及（加权）量化误差等。

In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.5], seasonality=2016)
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_data))
agg_metrics
{'MSE': 163.59102376302084,
 'abs_error': 1090.9220886230469,
 'abs_target_sum': 5658.0,
 'abs_target_mean': 52.38888888888889,
 'seasonal_error': 18.833625618877182,
 'MASE': 0.5361500323952336,
 'sMAPE': 0.21201368270827592,
 'MSIS': 21.446000940010823,
 'QuantileLoss[0.5]': 1090.9221000671387,
 'Coverage[0.5]': 0.34259259259259256,
 'RMSE': 12.790270668090681,
 'NRMSE': 0.24414090352665138,
 'ND': 0.19281054942082837,
 'wQuantileLoss[0.5]': 0.19281055144346743,
 'mean_wQuantileLoss': 0.19281055144346743,
 'MAE_Coverage': 0.15740740740740744}

## 方法比较
你可以将以上指标与其他模型或是你的预测应用的业务要求作比较。例如，我们可以用 Seasonal Naive Method 进行预测，然后与以上结果比较。Seasonal Naive Method 假设数据会有一个固定的周期性（本例中，2016 个数据点是一周，作为一个周期），并通过基于周期性来复制之前观察到的训练数据进行预测。

In [ ]:
from gluonts.model.seasonal_naive import SeasonalNaivePredictor

seasonal_predictor_1W = SeasonalNaivePredictor(freq="5min", prediction_length=36, season_length=2016)

forecast_it, ts_it = make_evaluation_predictions(test_data, predictor=seasonal_predictor_1W, num_samples=100)
forecasts = list(forecast_it)
tss = list(ts_it)

agg_metrics_seasonal, item_metrics_seasonal = evaluator(iter(tss), iter(forecasts), num_series=len(test_data))

df_metrics = pd.DataFrame.join(
    pd.DataFrame.from_dict(agg_metrics, orient='index').rename(columns={0: "DeepAR"}),
    pd.DataFrame.from_dict(agg_metrics_seasonal, orient='index').rename(columns={0: "Seasonal naive"})
)

df_metrics.loc[["MASE", "sMAPE", "RMSE"]]

通过比较以上这些指标，就可以得到你的模型与基线模型或其他高阶模型的对比。想要进一步提升模型性能的话，可以尝试改进模型架构，或者调节超参数的值。

# 时序异常检测方法案例
本例中使用DeepAR的方法对电力系统中的异常案例进行检测

In [ ]:
import numpy as np
from itertools import islice
from functools import partial
import mxnet as mx
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters

In [ ]:
register_matplotlib_converters()

from gluonts.dataset.loader import TrainDataLoader
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.util import get_hybrid_forward_input_names
from gluonts.mx.trainer import Trainer
from gluonts.mx.batchify import batchify
from gluonts.dataset.repository.datasets import get_dataset

In [ ]:
dataset = get_dataset(dataset_name="electricity")

In [ ]:
estimator = DeepAREstimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        trainer=Trainer(
            learning_rate=1e-3, epochs=20, num_batches_per_epoch=100
        )
    )

In [ ]:
train_output = estimator.train_model(dataset.train)
input_names = get_hybrid_forward_input_names(
        type(train_output.trained_net)
    )
batch_size = 500
num_samples = 100
instance_splitter = estimator._create_instance_splitter("training")
training_data_loader = TrainDataLoader(
    dataset=dataset.train,
     transform=train_output.transformation + instance_splitter,
    batch_size=batch_size,
    num_batches_per_epoch=estimator.trainer.num_batches_per_epoch,
    stack_fn=partial(
        batchify, ctx=estimator.trainer.ctx, dtype=estimator.dtype
    ),
)

data_entry = next(iter(training_data_loader))

# we now call the train model to get the predicted distribution on each window
# this allows us to investigate where are the biggest anomalies
context_length = train_output.trained_net.context_length
prediction_length = train_output.trained_net.prediction_length

distr = train_output.trained_net.distribution(
    *[data_entry[k] for k in input_names]
)

# gets all information into numpy array for further plotting
samples = distr.sample(num_samples).asnumpy()
percentiles = np.percentile(samples, axis=0, q=[10.0, 90.0])
target = mx.ndarray.concat(data_entry["past_target"], data_entry["future_target"], dim=1
)
target = target[:, -(context_length + prediction_length) :]
nll = -distr.log_prob(target).asnumpy()
target = target.asnumpy()
mean = samples.mean(axis=0)


In [ ]:
# NLL indices from largest to smallest
sorted_indices = np.argsort(nll.reshape(1,-1))[::-1]

# shows the series and times when the 20 largest NLL were observed
for k in sorted_indices[:20].reshape(-1):
    i = k // nll.shape[1]
    t = k % nll.shape[1]

    time_index = pd.date_range(start = pd.Timestamp(data_entry["forecast_start"][i]),periods = context_length + prediction_length)
        
    time_index -= context_length * time_index.freq

    plt.figure(figsize=(10, 4))
    plt.fill_between(
        time_index,
        percentiles[0, i],
        percentiles[-1, i],
        alpha=0.5,
        label="80% CI predicted"
    )
    plt.plot(time_index, target[i], label="target")
    plt.axvline(time_index[t], alpha=0.5, color="r")
    plt.title(f"NLL: {nll[i, t]}")
    plt.legend()
    plt.show()

In [ ]:
type(data_entry["forecast_start"][1])

In [ ]:
prediction_length

In [ ]:
nll.shape[1]

In [ ]:
nll.shape[1]

In [ ]:
time_index

In [ ]:
percentiles.shape

In [ ]:
sorted_indices[:20].reshape(-1)

In [ ]:
6316//48